<a href="https://colab.research.google.com/github/Choo-Wooo/Teeth/blob/main/teeth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import json
import pprint

# Load the entire JSON file into memory
with open('/content/drive/MyDrive/7.Data/train/_annotations.coco.json', 'r') as f:
    data = json.load(f)

# Assuming 'data' is a dictionary that contains the list you want to normalize under the key 'annotations'
# Adjust the key as per your JSON structure
df = pd.json_normalize(data, 'annotations')

print(df.info())

pprint.pp(data)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
                                    895,
                                    1962,
                                    873,
                                    2021,
                                    847,
                                    2062,
                                    824,
                                    2089,
                                    798,
                                    2117,
                                    760,
                                    2135,
                                    709,
                                    2146,
                                    667,
                                    2152,
                                    628,
                                    2153,
                                    621]],
                  'iscrowd': 0},
                 {'id': 822,
                  'image_id': 18,
                  'category_id': 35,
                  'bbox': [1736, 689, 525

데이터 전처리

In [58]:
import json
import os
from PIL import Image
import torch
from torchvision.transforms import functional as F

# 필요한 디렉토리를 생성하는 함수
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# JSON 파일을 읽고 데이터 파싱
def load_coco_annotations(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

# 주석 데이터에서 각 이미지에 대한 라벨 및 바운딩 박스 정보를 추출
def get_annotations_for_image(annotations, image_id):
    annotations_for_image = [ann for ann in annotations if ann['image_id'] == image_id]
    return annotations_for_image

# 이미지를 리사이즈하고, 주석 라벨을 해당 크기에 맞게 조정
def resize_image_and_annotations(image, annotations, new_size):
    # 이미지 리사이즈
    original_size = image.size
    image = F.resize(image, new_size)
    scale_x = new_size[0] / original_size[0]
    scale_y = new_size[1] / original_size[1]

    # 라벨 조정
    for annotation in annotations:
        bbox = annotation['bbox']
        bbox[0] *= scale_x
        bbox[1] *= scale_y
        bbox[2] *= scale_x
        bbox[3] *= scale_y
        annotation['bbox'] = bbox
    return image, annotations

# JSON 파일과 이미지 경로 설정
json_path = '/content/drive/MyDrive/7.Data/train/_annotations.coco.json'
images_dir = '/content/drive/MyDrive/7.Data/train/'
preprocess_dir = '/content/drive/MyDrive/7.Data/train/preprocess/'

# 새로 저장할 JSON 파일 준비
preprocessed_annotations = {
    "images": [],
    "annotations": [],
    "categories": []
}

# 데이터 로드
coco_data = load_coco_annotations(json_path)

# 전처리할 폴더 생성
create_dir_if_not_exists(preprocess_dir)

new_size = (512, 512)  # 원하는 새 이미지 크기

# 모든 이미지에 대한 전처리
for img_info in coco_data['images']:
    image_id = img_info['id']
    image_path = os.path.join(images_dir, img_info['file_name'])

    # 이미지 로드
    image = Image.open(image_path).convert('RGB')

    # 해당 이미지의 주석 정보 추출
    annotations = get_annotations_for_image(coco_data['annotations'], image_id)

    # 이미지와 주석 리사이즈
    resized_image, resized_annotations = resize_image_and_annotations(image, annotations, new_size)

    # 리사이즈된 이미지 저장
    resized_image_path = os.path.join(preprocess_dir, img_info['file_name'])
    resized_image.save(resized_image_path)

    # 이미지 정보와 주석을 전처리된 JSON 데이터에 추가
    img_info['file_name'] = os.path.join(img_info['file_name'])
    img_info['width'], img_info['height'] = new_size
    preprocessed_annotations['images'].append(img_info)
    preprocessed_annotations['annotations'].extend(resized_annotations)

# 카테고리 정보도 복사
preprocessed_annotations['categories'] = coco_data['categories']

# 전처리된 주석을 새 JSON 파일로 저장
with open(os.path.join(preprocess_dir, '_annotations.coco.json'), 'w') as f:
    json.dump(preprocessed_annotations, f)

# 전처리가 완료되었다는 메시지 출력
print('Preprocessing complete. Resized images and annotations saved to:', preprocess_dir)


Preprocessing complete. Resized images and annotations saved to: /content/drive/MyDrive/7.Data/train/preprocess/


라이브러리 선언

In [95]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
import torchvision.transforms as T
from torch.utils.data import DataLoader, Subset, random_split
from torch.utils.data.dataloader import default_collate
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Faster R-CNN 모델 구조 선언

In [96]:
# 모델 생성
def faster_rcnn(num_classes):
    # pre-trained 모델 로드
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # 분류기(classifier)의 입력 특징 수 얻기
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # pre-trained된 헤드를 새로운 것으로 교체
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

데이터셋 설정

In [97]:
# COCO 데이터셋을 로드하기 위한 함수
def get_coco(api_loc, path):
    # 데이터셋에 대한 변환(transformation) 정의
    transform = T.Compose([
        T.ToTensor(),
    ])

    # CocoDetection 클래스를 이용하여 데이터셋 로드
    dataset = CocoDetection(path, annFile=api_loc, transform=transform)
    return dataset

def collate_fn(batch):
    images, annotations = zip(*batch)
    images = list(img for img in images)  # 이미지 리스트

    targets = []
    for annots in annotations:
        boxes = [torch.tensor(annot['bbox'], dtype=torch.float32) for annot in annots]
        labels = [torch.tensor(annot['category_id'], dtype=torch.int64) for annot in annots]

        # Faster R-CNN은 x_min, y_min, x_max, y_max 형태의 boxes를 기대합니다.
        # COCO 데이터셋은 x_min, y_min, width, height 형태로 제공하므로 변환해야 합니다.
        boxes = [torch.tensor([box[0], box[1], box[0] + box[2], box[1] + box[3]], dtype=torch.float32) for box in boxes]

        targets.append({
            'boxes': torch.stack(boxes),
            'labels': torch.stack(labels)
        })

    return images, targets


하이퍼 파라미터 설정

In [101]:
# 하이퍼파라미터 딕셔너리
hyperparams = {
    'data_path': '/content/drive/MyDrive/7.Data/train/preprocess/',
    'num_classes': 42,
    'batch_size': 4,
    'num_epochs': 1,
    'learning_rate': 0.005,
    'momentum': 0.9,
    'weight_decay': 0.0005,
    'lr_step_size': 3,
    'lr_gamma': 0.1,
    'output_dir': '/path/to/save'
}

모델 결과 출력 코드

In [104]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchvision.ops import box_iou

# IoU 계산 및 시각화를 위한 함수 정의
def visualize_predictions(image, target, prediction, threshold=0.5):
    """
    이미지에 실제 및 예측된 바운딩 박스를 시각화하고 IoU를 계산합니다.
    """
    fig, ax = plt.subplots(1)
    ax.imshow(image.permute(1, 2, 0).cpu().numpy())

    for box in target['boxes']:
        rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=1, edgecolor='g', facecolor='none')
        ax.add_patch(rect)

    for box in prediction['boxes']:
        if box[4] >= threshold:
            rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

    plt.show()


모델 학습 코드

In [105]:
# 데이터셋 로드
transform = T.Compose([T.ToTensor()])
dataset = CocoDetection(hyperparams['data_path'], hyperparams['data_path'] + '/_annotations.coco.json', transform=transform)

# 데이터셋을 train과 test로 분할
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=hyperparams['batch_size'], shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

# 모델을 가져옵니다.
model = faster_rcnn(hyperparams['num_classes'])

# 옵티마이저와 스케쥴러 설정
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=hyperparams['learning_rate'], momentum=hyperparams['momentum'], weight_decay=hyperparams['weight_decay'])
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=hyperparams['lr_step_size'], gamma=hyperparams['lr_gamma'])

# 학습 장치 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# 학습 루프
for epoch in range(hyperparams['num_epochs']):
    # 훈련 모드
    model.train()
    train_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{hyperparams['num_epochs']} [Training]")
    for images, targets in progress_bar:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        train_loss += losses.item()

        # 역전파
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        # tqdm에 평균 손실을 업데이트하여 출력합니다.
        progress_bar.set_postfix(loss=f"{train_loss / (progress_bar.n + 1):.4f}")

    # 학습률 감소
    lr_scheduler.step()

# 정확도 및 평균 IoU 계산
iou_threshold = 0.5
total_iou = []
accurate_detections = 0
total_detections = 0

model.eval()
with torch.no_grad():
    for images, targets in test_loader:
        images = list(img.to(device) for img in images)
        targets = [{k: torch.as_tensor(v, dtype=torch.float32, device=device) for k, v in t.items()} for t in targets]
        outputs = model(images)

        for target, output in zip(targets, outputs):
            # IoU 계산
            ious = box_iou(target['boxes'], output['boxes'][:, :4])
            max_iou, _ = ious.max(dim=1)
            total_iou.extend(max_iou.cpu().numpy())

            # 정확도 계산 (IoU > iou_threshold)
            accurate_detections += (max_iou >= iou_threshold).sum().item()
            total_detections += len(max_iou)

        # 예시 이미지에 대한 시각화
        visualize_predictions(images[0], targets[0], output)

# 평균 IoU와 정확도 계산
mean_iou = np.mean(total_iou)
accuracy = accurate_detections / total_detections

print(f"Mean IoU: {mean_iou:.4f}")
print(f"Accuracy (IoU > {iou_threshold}): {accuracy:.4f}")

# 모델 저장 경로 설정
model_save_path = os.path.join(hyperparams['output_dir'], 'faster_rcnn.pth')
torch.save(model.state_dict(), model_save_path)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


Epoch 1/1 [Training]: 100%|██████████| 4/4 [04:41<00:00, 70.38s/it, loss=5.4834]


IndexError: max(): Expected reduction dim 1 to have non-zero size.